In [1]:
import pandas as pd
import numpy as np
import math
from scipy.stats import wilcoxon

In [2]:
def regulate_metrics(values, lenght):
    if len(values) != lenght:

        size_of_batch = len(values) / lenght
        if type(size_of_batch) == float:
            size_of_batch = math.ceil(size_of_batch)
            pad = size_of_batch * lenght
            values = np.pad(values, (pad - len(values)))


        new_values = []
        old_i = 0
        for i in range(size_of_batch, len(values), size_of_batch):
            new_values.append(np.sum(values[old_i:i]))
            old_i = i
        

    return new_values

In [3]:
def load_error_and_rank_metrics(algorithms, dataset, emb, lenght, path="results/"):

    labels = ['MSE', 'MAE', 'RMSE', 'PRECISION', 
              'RECALL', 'DIVERSITY', 'NOVELTY', 'NDCG']
    
    alg_name_1 = [""]*len(labels) 
    alg_value_1 = [1.0]*len(labels[:3]) + [0.0]*len(labels[3:])

    alg_name_2 = [""]*len(labels)
    alg_value_2 = [1.0]*len(labels[:3]) + [0.0]*len(labels[3:])

    draw = []
    metrics_matrix = []
    
    for algorithm in algorithms:
            error_metrics = pd.read_csv(
                path + algorithm + "_" + dataset + "_" + emb + "_results_error.csv")
            
            rank_metrics = pd.read_csv(
                path + algorithm + "_" + dataset + "_" + emb + "_results_rank.csv")
            

            values = np.concatenate([error_metrics.mean().values, 
                                    rank_metrics.mean().values])
            
            for i, m in enumerate(values[:3]):
                if m < alg_value_1[i]:
                    alg_name_1[i] = algorithm
                    alg_value_1[i] = m

                elif m < alg_value_2[i]:
                    alg_name_2[i] = algorithm
                    alg_value_2[i] = m

            for i, m in enumerate(values[3:]):
                idx = i + 3
                if m > alg_value_1[idx]:
                    alg_name_1[idx] = algorithm
                    alg_value_1[idx] = m

                elif m > alg_value_2[idx]:
                    alg_name_2[idx] = algorithm
                    alg_value_2[idx] = m


            metrics_matrix.append(values)

    # for i, (alg1, alg2) in enumerate(zip(alg_name_1, alg_name_2)):
        
    #     alg1_metrics_error = pd.read_csv(
    #             path + alg1 + "_" + dataset + "_" + emb + "_results_error.csv").values.transpose()
            
    #     alg1_metrics_rank = pd.read_csv(
    #             path + alg1 + "_" + dataset + "_" + emb + "_results_rank.csv").values.transpose()
        
    #     alg2_metrics_error = pd.read_csv(
    #             path + alg2 + "_" + dataset + "_" + emb + "_results_error.csv").values.transpose()
            
    #     alg2_metrics_rank = pd.read_csv(
    #             path + alg2 + "_" + dataset + "_" + emb + "_results_rank.csv").values.transpose()
        
    #     if i < 7:
    #         alg1_metric = alg1_metrics_error[i]
    #         alg2_metric = alg2_metrics_error[i]
    #     else:
    #         alg1_metric = alg1_metrics_rank[0]
    #         alg2_metric = alg2_metrics_rank[0]

    #     if len(alg1_metric) > lenght:
    #         alg1_metric = regulate_metrics(alg1_metric, lenght)
    #     if len(alg2_metric) > lenght:
    #         alg2_metric = regulate_metrics(alg2_metric, lenght)

    #     if not (np.array(alg1_metric) - np.array(alg2_metric)).any():
    #         test = 0.0
    #     else:
    #         test = wilcoxon(alg1_metric, alg2_metric).pvalue
            
    #     if test < 0.05:
    #         draw.append('Draw')
    #     else:
    #         draw.append('Winner')


    df = pd.DataFrame(np.array(metrics_matrix),
                      columns=labels, index=algorithms)
    
    # rank = pd.DataFrame({"Metric": labels,
    #                     "Winner":alg_name_1,
    #                      "2 Place": alg_name_2,
    #                      "Draw": draw})
    rank = []

    return df.round(3), rank
    
    # return  error_metrics.mean(), rank_metrics.mean()

In [154]:
algorithms = ["DeepCoNN",
              "ConvMF", "TARMF", "CARM"]


df, draw = load_error_and_rank_metrics(algorithms, "AMAZON_FASHION_data", "word2vec", 3)

/home/guilherme/iRev/.venv/lib/python3.10/site-packages/scipy/stats/_morestats.py:4102: UserWarning: Sample size too small for normal approximation.
  warnings.warn("Sample size too small for normal approximation.")


In [18]:
algorithms = ["DeepCoNN", "ConvMF", "NARRE", "D_ATTN", "MPCN",
            "ALFM", "TRANSNET", "A3NCF", "TARMF", "ANR",
            "CARL", "DAML", "CARP", "CARM", "HRDR"]


df, draw = load_error_and_rank_metrics(algorithms, "Digital_Music_data", "word2vec", 133)
df.transpose()

,DeepCoNN,ConvMF,NARRE,D_ATTN,MPCN,ALFM,TRANSNET,A3NCF,TARMF,ANR,CARL,DAML,CARP,CARM,HRDR
MSE,0.347,0.411,0.321,0.347,0.405,0.493,0.493,0.493,0.376,0.379,0.324,0.382,0.302,0.349,0.320
MAE,0.384,0.452,0.345,0.347,0.421,0.479,0.474,0.472,0.461,0.373,0.336,0.403,0.316,0.371,0.336
RMSE,0.583,0.634,0.559,0.583,0.630,0.694,0.694,0.694,0.608,0.608,0.562,0.541,0.542,0.583,0.559
PRECISION,0.463,0.525,0.497,0.481,0.525,0.799,0.799,0.799,0.460,0.489,0.502,0.685,0.511,0.451,0.498
RECALL,0.319,0.274,0.345,0.304,0.261,0.224,0.224,0.224,0.339,0.286,0.331,0.614,0.366,0.303,0.333
DIVERSITY,0.356,0.354,0.362,0.369,0.360,0.358,0.360,0.360,0.371,0.370,0.371,0.380,0.365,0.361,0.368
NOVELTY,0.061,0.071,0.043,0.043,0.048,0.053,0.053,0.053,0.155,0.049,0.046,0.175,0.041,0.044,0.040
NDCG,0.000,0.000,0.016,0.024,0.040,0.016,0.000,0.024,0.032,0.024,0.016,0.000,0.024,0.032,0.032


In [152]:
draw

,Metric,Winner,2 Place,Draw
0,MSE,CARP,HRDR,Draw
1,MAE,CARP,HRDR,Draw
2,RMSE,DAML,CARP,Draw
3,PRECISION,ALFM,TRANSNET,Draw
4,RECALL,DAML,CARP,Draw
5,DIVERSITY,DAML,ANR,Draw
6,NOVELTY,DAML,ALFM,Draw
7,NDCG,MPCN,TARMF,Winner


In [108]:
daml = pd.read_csv("results/DAML_Digital_Music_data_word2vec_results_error.csv")
daml_recall = daml['recall'].values

narre = pd.read_csv("results/NARRE_Digital_Music_data_word2vec_results_error.csv")
narre_recall = narre['recall'].values

In [110]:
wilcoxon(regulate_metrics(daml_recall, 133), narre_recall)

WilcoxonResult(statistic=0.0, pvalue=1.4317759392997278e-23)

In [18]:
pd.read_csv("results/DeepCoNN_Tucso_data_word2vec_results_error.csv").shape

(254, 7)

In [19]:
df, draw = load_error_and_rank_metrics(algorithms, "Musical_Instruments_data", "word2vec", 181)
df.transpose()

,DeepCoNN,ConvMF,NARRE,D_ATTN,MPCN,ALFM,TRANSNET,A3NCF,TARMF,ANR,CARL,DAML,CARP,CARM,HRDR
MSE,0.884,0.927,0.862,0.889,0.922,0.841,0.977,0.977,0.882,0.903,0.854,0.925,0.884,0.885,0.860
MAE,0.667,0.753,0.670,0.685,0.718,0.637,0.747,0.746,0.720,0.650,0.658,0.716,0.668,0.666,0.653
RMSE,0.935,0.959,0.924,0.939,0.955,0.912,0.983,0.983,0.935,0.945,0.919,0.887,0.935,0.936,0.923
PRECISION,0.442,0.507,0.414,0.402,0.509,0.535,0.164,0.164,0.386,0.497,0.455,0.491,0.434,0.385,0.417
RECALL,0.243,0.222,0.256,0.247,0.234,0.262,0.200,0.200,0.244,0.239,0.256,0.386,0.239,0.245,0.251
DIVERSITY,0.206,0.297,0.240,0.237,0.216,0.206,0.213,0.210,0.316,0.162,0.213,0.232,0.195,0.208,0.209
NOVELTY,0.070,0.119,0.075,0.075,0.072,0.069,0.074,0.074,0.134,0.106,0.069,0.281,0.074,0.069,0.069
NDCG,0.023,0.008,0.023,0.023,0.023,0.016,0.000,0.000,0.016,0.023,0.023,0.000,0.023,0.031,0.023


In [155]:
draw

,Metric,Winner,2 Place,Draw
0,MSE,ALFM,CARL,Draw
1,MAE,ALFM,ANR,Draw
2,RMSE,DAML,CARL,Draw
3,PRECISION,ALFM,ANR,Draw
4,RECALL,DAML,CARL,Draw
5,DIVERSITY,TARMF,NARRE,Draw
6,NOVELTY,DAML,ANR,Draw
7,NDCG,CARM,NARRE,Winner


In [21]:
algorithms = ["DeepCoNN", "ConvMF", "NARRE", "D_ATTN", "MPCN",
            "ALFM", "TRANSNET", "A3NCF", "TARMF", "ANR",
            "CARL", "DAML", "CARP", "CARM"]

df, draw = load_error_and_rank_metrics(algorithms, "Office_Products_data", "word2vec", 625)
df.transpose()

,DeepCoNN,ConvMF,NARRE,D_ATTN,MPCN,ALFM,TRANSNET,A3NCF,TARMF,ANR,CARL,DAML,CARP,CARM
MSE,0.933,0.984,0.922,0.928,0.970,1.028,1.028,1.028,0.922,0.946,0.943,1.002,0.930,0.927
MAE,0.687,0.731,0.676,0.664,0.706,0.754,0.754,0.755,0.717,0.709,0.707,0.741,0.676,0.701
RMSE,0.961,0.986,0.955,0.958,0.979,1.008,1.008,1.008,0.956,0.967,0.966,0.921,0.959,0.958
PRECISION,0.399,0.474,0.379,0.385,0.458,0.152,0.152,0.152,0.371,0.466,0.490,0.461,0.373,0.337
RECALL,0.244,0.226,0.248,0.243,0.231,0.202,0.202,0.202,0.248,0.236,0.246,0.350,0.240,0.255
DIVERSITY,0.201,0.194,0.189,0.163,0.175,0.181,0.181,0.182,0.275,0.201,0.202,0.195,0.164,0.250
NOVELTY,0.075,0.074,0.070,0.071,0.071,0.073,0.073,0.073,0.129,0.075,0.076,0.286,0.071,0.106
NDCG,0.023,0.023,0.023,0.016,0.008,0.016,0.010,0.016,0.016,0.008,0.016,0.000,0.016,0.039


In [173]:
draw

,Metric,Winner,2 Place,Draw
0,MSE,TARMF,CARM,Draw
1,MAE,D_ATTN,CARP,Draw
2,RMSE,DAML,TARMF,Draw
3,PRECISION,CARL,ANR,Draw
4,RECALL,DAML,CARM,Draw
5,DIVERSITY,TARMF,CARM,Draw
6,NOVELTY,DAML,CARM,Draw
7,NDCG,CARM,ConvMF,Winner


In [13]:
df, draw = load_error_and_rank_metrics(algorithms, "Tamp_data", "word2vec", 264)
df.transpose()

,DeepCoNN,ConvMF,NARRE,D_ATTN,MPCN,ALFM,TRANSNET,A3NCF,TARMF,ANR,CARL,DAML,CARP,CARM
MSE,1.696,1.647,1.631,1.715,1.791,1.533,1.756,1.962,1.702,1.657,1.720,1.737,1.600,1.710
MAE,1.008,1.056,0.988,1.038,1.081,0.997,1.082,1.184,1.007,1.002,1.041,1.088,0.977,1.021
RMSE,1.299,1.281,1.274,1.307,1.336,1.236,1.323,1.399,1.302,1.284,1.309,1.285,1.262,1.305
PRECISION,0.359,0.347,0.372,0.379,0.322,0.313,0.288,0.230,0.384,0.318,0.318,0.266,0.330,0.380
RECALL,0.276,0.239,0.276,0.280,0.235,0.255,0.222,0.200,0.284,0.251,0.253,0.261,0.262,0.272
DIVERSITY,0.120,0.045,0.133,0.036,0.082,0.103,0.120,0.067,0.085,0.192,0.116,0.112,0.184,0.087
NOVELTY,0.094,0.141,0.093,0.156,0.107,0.091,0.095,0.120,0.103,0.152,0.096,0.384,0.140,0.105
NDCG,0.016,0.016,0.008,0.016,0.008,0.008,0.000,0.016,0.016,0.016,0.016,0.000,0.016,0.008


In [15]:
df, _ = load_error_and_rank_metrics(algorithms, "Philladelphi_data", "word2vec", 610)
df.transpose()

,DeepCoNN,ConvMF,NARRE,D_ATTN,MPCN,ALFM,TRANSNET,A3NCF,TARMF,ANR,CARL,DAML,CARP,CARM
MSE,1.487,1.416,1.412,1.473,1.586,1.372,1.711,1.711,1.479,1.450,1.475,1.545,1.401,1.450
MAE,0.952,0.952,0.923,0.941,0.995,0.926,1.076,1.075,0.932,0.932,0.951,1.010,0.933,0.923
RMSE,1.217,1.188,1.185,1.211,1.256,1.169,1.306,1.306,1.213,1.201,1.212,1.209,1.181,1.201
PRECISION,0.358,0.346,0.365,0.364,0.322,0.400,0.280,0.280,0.377,0.340,0.339,0.286,0.335,0.363
RECALL,0.261,0.242,0.267,0.254,0.225,0.255,0.200,0.200,0.272,0.249,0.247,0.275,0.262,0.268
DIVERSITY,0.208,0.197,0.225,0.250,0.224,0.238,0.173,0.175,0.240,0.300,0.245,0.215,0.222,0.275
NOVELTY,0.085,0.084,0.084,0.097,0.092,0.089,0.098,0.098,0.091,0.137,0.095,0.350,0.083,0.110
NDCG,0.008,0.008,0.000,0.008,0.008,0.008,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.008


In [11]:
df, _ = load_error_and_rank_metrics(algorithms, "Tucso_data", "word2vec", 254)
df = df.fillna(0.006).round(3)
df['DIVERSITY'] = df['DIVERSITY'] * -1

In [12]:
df.transpose()

,DeepCoNN,ConvMF,NARRE,D_ATTN,MPCN,ALFM,TRANSNET,A3NCF,TARMF,ANR,CARL,DAML,CARP,CARM
MSE,1.849,1.840,1.786,1.847,1.964,1.700,2.156,2.202,1.856,1.793,1.844,1.925,1.772,1.858
MAE,1.078,1.144,1.060,1.073,1.162,1.057,1.239,1.269,1.081,1.091,1.107,1.168,1.078,1.089
RMSE,1.357,1.355,1.333,1.356,1.399,1.301,1.467,1.482,1.359,1.337,1.355,1.359,1.328,1.360
PRECISION,0.349,0.321,0.365,0.354,0.312,0.379,0.134,0.206,0.380,0.355,0.367,0.235,0.337,0.362
RECALL,0.272,0.237,0.275,0.265,0.231,0.252,0.208,0.200,0.286,0.259,0.271,0.237,0.265,0.280
DIVERSITY,0.046,0.098,0.043,0.022,0.066,0.034,0.073,0.056,0.094,0.062,0.104,0.021,0.066,0.093
NOVELTY,0.102,0.139,0.100,0.120,0.104,0.106,0.111,0.108,0.134,0.095,0.155,0.392,0.097,0.133
NDCG,0.008,0.008,0.000,0.000,0.008,0.000,0.006,0.008,0.000,0.000,0.008,0.000,0.008,0.008
